In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor

In [2]:
area_feature_training_sample = pd.read_csv('../../data/pre_training/area_feature_training_sample.csv')
area_target_training_sample = pd.read_csv('../../data/pre_training/area_target_training_sample.csv')
district_feature_training_sample = pd.read_csv('../../data/pre_training/district_feature_training_sample.csv')
district_target_training_sample = pd.read_csv('../../data/pre_training/district_target_training_sample.csv')

In [ ]:
area_feature_testing_data = pd.read_csv('../../data/pre_training/area_feature_testing_data.csv')
area_target_testing_data = pd.read_csv('../../data/pre_training/area_target_testing_data.csv')
district_feature_testing_data = pd.read_csv('../../data/pre_training/district_feature_testing_data.csv')
district_target_testing_data = pd.read_csv('../../data/pre_training/district_target_testing_data.csv')

In [ ]:
area_feature_training_sample.drop('crime_status', axis=1, inplace=True)
district_feature_training_sample.drop('crime_status', axis=1, inplace=True)

In [ ]:
area_dtrain_reg = xgb.DMatrix(area_feature_training_sample, area_target_training_sample, enable_categorical=True)
area_dtest_reg = xgb.DMatrix(area_feature_testing_data, area_target_testing_data, enable_categorical=True)

In [ ]:
district_dtrain_reg = xgb.DMatrix(district_feature_training_sample, district_target_training_sample, enable_categorical=True)
district_dtest_reg = xgb.DMatrix(district_feature_testing_data, district_target_testing_data, enable_categorical=True)

In [ ]:
class Evaluator:
    def __init__(self, model, testing):
        self.model = model
        self.test_features = testing
        self.sample_size = len(testing.get_label())
        self.independent_vars = len(testing.feature_names)
        self.predictions = None
        self.residuals = None

    def gather_predictions(self):
        self.predictions = self.model.predict(self.test_features)
        self.residuals = self.test_features.get_label() - self.predictions
    
    def mae(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        return np.mean(np.abs(self.residuals))
    
    def mse(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        return np.mean(self.residuals ** 2)
    
    def rmse(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        return np.sqrt(self.mse())
    
    def relative_rmse(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        return self.rmse() / max(self.test_features.get_label())

    def r_squared(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        ss_res = np.sum(self.residuals ** 2)
        ss_tot = np.sum((self.test_features.get_label() - np.mean(self.test_features.get_label())) ** 2)
        return 1 - (ss_res / ss_tot)

    def adjusted_r_squared(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        r2 = self.r_squared()
        return 1 - (1 - r2) * ((self.sample_size - 1) / (self.sample_size - self.independent_vars - 1))
    
    def median_absolute_error(self):
        if self.residuals is None:
            return "Predictions need to be gathered first"
        return np.median(np.abs(self.residuals))

    def feature_importances(self):
        # Get feature importance based on different metrics
        importance_weight = self.model.get_score(importance_type='weight')  # number of times a feature is used to split the data
        importance_gain = self.model.get_score(importance_type='gain')  # improvement in accuracy of a feature to the branches it's on
        importance_cover = self.model.get_score(importance_type='cover')  # average number of samples affected by a feature

        # Sort and print feature importance based on weight
        sorted_weight = sorted(importance_weight.items(), key=lambda x: x[1], reverse=True)
        print("Feature importance based on weight (sorted):", sorted_weight)

        # Sort and print feature importance based on gain
        sorted_gain = sorted(importance_gain.items(), key=lambda x: x[1], reverse=True)
        print("Feature importance based on gain (sorted):", sorted_gain)

        # Sort and print feature importance based on cover
        sorted_cover = sorted(importance_cover.items(), key=lambda x: x[1], reverse=True)
        print("Feature importance based on cover (sorted):", sorted_cover)

        return importance_weight, importance_gain, importance_cover

    def plot_residuals(self, title, save_name):
        # Calculate percent differences
        percent_diff = (self.residuals / max(self.test_features.get_label())) * 100
        
        plt.figure(figsize=(10, 5))
        plt.plot(percent_diff, label='Percent Difference', marker='o')
        
        # Calculate the average percent difference and plot it
        avg_percent_diff = np.mean(percent_diff)
        plt.axhline(y=avg_percent_diff, color='r', linestyle='dashed', label='Average Percent Difference')
        
        plt.text(0, avg_percent_diff + 1, f'Avg: {avg_percent_diff:.5f}%', color='r', fontsize=12)

        plt.title(f'Percent Difference between Test Labels and Predicted Values - {title} Model')
        plt.xlabel('Sample Index')
        plt.ylabel('Percent Difference (%)')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'../results/xgboost/{save_name}.png')
        plt.show()

In [ ]:
def evaluate(model, test_labels, title=None, save_name=None):
    ev = Evaluator(model, test_labels)
    ev.gather_predictions()
    
    print(f"Mean Absolute Error (MAE): {ev.mae():.4f}")
    print(f"Mean Squared Error (MSE): {ev.mse():.4f}")
    print(f"Root Mean Squared Error (RMSE): {ev.rmse():.4f}")
    print(f"Relative Root Mean Squared Error (Relative RMSE): {ev.relative_rmse():.4f}")
    print(f"R-squared (R²): {ev.r_squared():.4f}")
    print(f"Adjusted R-squared: {ev.adjusted_r_squared():.4f}")
    print(f"Median Absolute Error: {ev.median_absolute_error():.4f}")
    
    importance_weight, importance_gain, importance_cover = ev.feature_importances()

    if title is not None:
        ev.plot_residuals(title, save_name)

    return importance_weight, importance_gain, importance_cover

##### Training the Area XGBoost Model

In [ ]:
area_params = {"objective": "reg:squarederror", "device":"gpu", "eta":"0.001"}
area_evals = [(area_dtest_reg, "validation"), (area_dtrain_reg, "train")]

In [ ]:
area_model = xgb.train(
   params=area_params,
   dtrain=area_dtrain_reg,
   num_boost_round=10000000,
   evals=area_evals,
   verbose_eval=1000,
   early_stopping_rounds=100
)

In [ ]:
area_importance_weight, area_importance_gain, area_importance_cover = evaluate(area_model, area_dtest_reg, 'Area', 'area_residuals')

##### Training the District XGBoost Model

In [ ]:
district_params = {"objective": "reg:squarederror", "device":"gpu", "eta":"0.001"}
district_evals = [(district_dtest_reg, "validation"), (district_dtrain_reg, "train")]

In [ ]:
district_model = xgb.train(
   params=district_params,
   dtrain=district_dtrain_reg,
   num_boost_round=10000000,
   evals=district_evals,
   verbose_eval=1000,
   early_stopping_rounds=100
)

In [ ]:
district_importance_weight, district_importance_gain, district_importance_cover = evaluate(district_model, district_dtest_reg, 'District', 'district_residuals')

##### Feature Selection

In [ ]:
def normalize_importances(importances):
    total = sum(importances.values())    
    return {key : val / total for key, val in importances.items()}

In [ ]:
def extract_features(importances, type, threshold):
    if type == 'cum_sum':
        sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)
        cumulative_sum = 0
        out = []
        for feature, importance in sorted_features:
            cumulative_sum += importance
            out.append(feature)
            if cumulative_sum >= threshold:
                break
        return out
    else:
        return [key for key, val in importances.items() if val >= threshold]

In [ ]:
def feature_selection(model, type, training_sample, testing_data, thresholds):
    weight_importances = normalize_importances(model.get_score(importance_type='weight'))
    gain_importances = normalize_importances(model.get_score(importance_type='gain'))
    cover_importances = normalize_importances(model.get_score(importance_type='cover'))    
    agg_importances = {feature : np.mean((weight_importances[feature], gain_importances[feature], cover_importances[feature])) for feature in weight_importances.keys()}
    
    for threshold in thresholds:
        target_features = extract_features(agg_importances, type, threshold)
        temp_dtrain = xgb.DMatrix(training_sample[0][target_features], training_sample[1], enable_categorical=True)
        temp_dtest = xgb.DMatrix(testing_data[0][target_features], testing_data[1], enable_categorical=True)
        temp_model = xgb.train(
            params={"objective": "reg:squarederror", "device":"gpu", "eta":"0.001"},
            dtrain=temp_dtrain,
            num_boost_round=10000000,
            evals=[(temp_dtest, "validation"), (temp_dtrain, "train")],
            verbose_eval=False,
            early_stopping_rounds=100
        )
        print(f'Threshold of {threshold} resulted in the features: {target_features}')
        evaluate(temp_model, temp_dtest)

In [ ]:
feature_selection(district_model, 'cum_sum', (district_feature_training_sample, district_target_training_sample), (district_feature_testing_data, district_target_testing_data), np.arange(0.9, 1, 0.02))

In [ ]:
feature_selection(district_model, 'cum_sum', (district_feature_training_sample, district_target_training_sample), (district_feature_testing_data, district_target_testing_data), np.arange(0.92, 1, 0.02))

In [ ]:
feature_selection(area_model, 'cum_sum', (area_feature_training_sample, area_target_training_sample), (area_feature_testing_data, area_target_testing_data), np.arange(0.9, 1, 0.02))

In [ ]:
feature_selection(area_model, 'cum_sum', (area_feature_training_sample, area_target_training_sample), (area_feature_testing_data, area_target_testing_data), np.arange(0.88, 1, 0.04))

##### Hyperparameter Tuning

In [ ]:
param_grid_district = {
    'n_estimators': [10000],
    'max_depth': np.arange(3, 20, 2),  # Explore deeper trees for potential complex patterns
    'min_child_weight': np.arange(1, 20, 2),  # Allow for more conservative and less conservative splits
    'gamma': np.linspace(0, 1.0, 10),  # Increase gamma range to explore more conservative split criteria
    'subsample': np.linspace(0.3, 1.0, 10),  # Lower the minimum subsample to add more randomness
    'colsample_bytree': np.linspace(0.3, 1.0, 10),  # Lower the minimum colsample_bytree to add more randomness
    'eta': [0.01],
    'objective':['reg:squarederror'],
    'device':['gpu']
}

In [ ]:
model_district = XGBRegressor()

In [ ]:
random_search_district = RandomizedSearchCV(
    estimator=model_district,
    param_distributions=param_grid_district,
    n_iter=200,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=3
)

In [ ]:
random_search_district.fit(district_feature_training_sample, district_target_training_sample)

In [ ]:
print("Best Parameters for District Model:", random_search_district.best_params_)
print("Best Score for District Model:", random_search_district.best_score_)

In [ ]:
param_grid_district = {
    'n_estimators': [10000],
    'max_depth': [19, 21, 23, 25, None],
    'min_child_weight': [1.0], 
    'gamma': [0.0], 
    'subsample': [0.8, 0.85, 0.9],
    'colsample_bytree': [0.65, 0.7, 0.75], 
    'eta': [0.01],
    'objective': ['reg:squarederror'],
    'device': ['gpu']
}

In [ ]:
model_district = XGBRegressor()

In [ ]:
grid_search_district = GridSearchCV(
    estimator=model_district,
    param_grid=param_grid_district,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=3
)

In [ ]:
grid_search_district.fit(district_feature_training_sample, district_target_training_sample)

In [ ]:
model_area = XGBRegressor()

In [ ]:
param_grid_area = {
    'n_estimators': [20000],
    'max_depth': np.arange(3, 20, 2),
    'min_child_weight': np.arange(1, 20, 2),
    'gamma': np.linspace(0, 1.0, 10),
    'subsample': np.linspace(0.3, 1.0, 10), 
    'colsample_bytree': np.linspace(0.3, 1.0, 10),
    'eta': [0.01],
    'objective':['reg:squarederror'],
    'device':['gpu']
}

In [ ]:
random_search_area = RandomizedSearchCV(
    estimator=model_area,
    param_distributions=param_grid_area,
    n_iter=200,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=3
)

In [ ]:
random_search_area.fit(area_feature_training_sample, area_target_training_sample)

print("Best Parameters for Area Model:", random_search_area.best_params_)
print("Best Score for Area Model:", random_search_area.best_score_)